In [21]:
from scsnp.snp_10x import quantify_SNP_in_position, annotate_snp_2_adata
import scanpy as sc
import pysam

download some sample data from 10x

In [4]:
!mkdir data
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_filtered_feature_bc_matrix.tar.gz -O data/pbmc_1k_protein_v3_filtered_feature_bc_matrix.tar.gz
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_possorted_genome_bam.bam -O data/pbmc_1k_protein_v3_possorted_genome_bam.bam
!wget http://cf.10xgenomics.com/samples/cell-exp/3.0.0/pbmc_1k_protein_v3/pbmc_1k_protein_v3_possorted_genome_bam.bam.bai -O data/pbmc_1k_protein_v3_possorted_genome_bam.bam.bai

In [7]:
!cd data; tar -xzf pbmc_1k_protein_v3_filtered_feature_bc_matrix.tar.gz

check a single locus inside CD8B. For each cell that has read coverage, it returns the bases detected

In [18]:
quantify_SNP_in_position(chrom='2', pos=86815701, bamfile=pysam.AlignmentFile('data/pbmc_1k_protein_v3_possorted_genome_bam.bam'))

defaultdict(list,
            {'TTCCACGCATCCAATG-1': ['T', 'C'],
             'GATGACTGTTAGGGTG-1': ['T'],
             'CTCTGGTTCCGAGGCT-1': ['C', 'C'],
             'ACTATCTGTGCAATAA-1': ['T'],
             'TCCACCATCAGCTGTA-1': ['C'],
             'AGAACAATCGCCAGAC-1': ['T'],
             'CATGGATGTCTGTGCG-1': ['C', 'C'],
             'GCTGGGTGTACGATTC-1': ['T'],
             'TCCATCGCAATTGTGC-1': ['C'],
             'GCGATCGGTGGAGGTT-1': ['C'],
             'CACGAATGTACGATCT-1': ['T'],
             'CAGGCCATCGTTCGCT-1': ['T']})

---
scanpy, annotating that locus to the dataset

In [19]:
adata = sc.read_10x_mtx('data/filtered_feature_bc_matrix/')

In [24]:
annotate_snp_2_adata(adata, '2', 86815701, 
                     pysam.AlignmentFile('data/pbmc_1k_protein_v3_possorted_genome_bam.bam'),
                     refbase='T')

Trying to set attribute `.obs` of view, making a copy.


Cells not in the adata:
TCCACCATCAGCTGTA-1 AGAACAATCGCCAGAC-1 CACGAATGTACGATCT-1


In [30]:
adata.obs.head()

,SNP_2_86815701_GT,SNP_2_86815701_nALT,SNP_2_86815701_coverage
0,,,
AAACCCAAGTGGTCAG-1,.,0,0
AAAGGTATCAACTACG-1,.,0,0
AAAGTCCAGCGTGTCC-1,.,0,0
AACACACTCAAGAGTA-1,.,0,0
AACACACTCGACGAGA-1,.,0,0


We get per cell:
- the 'inferred' genotype of all cells. `.` indicates no coverage
- the number of reads supporting an ALT allele
- the number of reads covering the SNP

only checking cells with coverage

In [31]:
adata.obs.query('SNP_2_86815701_GT !="."')

,SNP_2_86815701_GT,SNP_2_86815701_nALT,SNP_2_86815701_coverage
0,,,
ACTATCTGTGCAATAA-1,GT 0/0,0,1
CAGGCCATCGTTCGCT-1,GT 0/0,0,1
CATGGATGTCTGTGCG-1,GT 0/1,2,2
CTCTGGTTCCGAGGCT-1,GT 0/1,2,2
GATGACTGTTAGGGTG-1,GT 0/0,0,1
GCGATCGGTGGAGGTT-1,GT 0/1,1,1
GCTGGGTGTACGATTC-1,GT 0/0,0,1
TCCATCGCAATTGTGC-1,GT 0/1,1,1
TTCCACGCATCCAATG-1,GT 0/1,1,2
